In [1]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)

Welcome to ROOTaaS 6.06/05


<module 'training_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.pyc'>

In [2]:
ntuples = '20170509'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
signal = [s for s in files if "NonRes" in s]
background = [s for s in files if "DiPhotonJets" in s]

utils.IO.set_signal_and_background(ntuples,signal,background)

print "using background file:"+utils.IO.backgroundName
print "using signal file:"+utils.IO.signalName

env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509
using background file:/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using signal file:/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509/AllNonResSig.root


In [8]:
#normal variables: int, float, vectors...
branch_names = 'leadingJet_bDis,subleadingJet_bDis,CosThetaStar_CS,CosTheta_bb,CosTheta_gg'.split(",")
branch_names = [c.strip() for c in branch_names]
branch_names = (b.replace(" ", "_") for b in branch_names)
branch_names = list(b.replace("-", "_") for b in branch_names)

#tlorentz vectors
branch_names_lorentz = 'didiphotonCandidate,dijetCandidate,diHiggsCandidate'.split(",")
branch_names_lorentz = [c.strip() for c in branch_names_lorentz]
branch_names_lorentz = (b.replace(" ", "_") for b in branch_names_lorentz)
branch_names_lorentz = list(b.replace("-", "_") for b in branch_names_lorentz)
print branch_names_lorentz

#variables we want to access in tlorenz vector
branch_names_lorentz_var = 'Pt(),M()'.split(",")
branch_names_lorentz_var = [c.strip() for c in branch_names_lorentz_var]
branch_names_lorentz_var = (b.replace(" ", "_") for b in branch_names_lorentz_var)
branch_names_lorentz_var = list(b.replace("-", "_") for b in branch_names_lorentz_var)
print branch_names_lorentz_var

['diphotonCandidate', 'dijetCandidate', 'diHiggsCandidate']
['Pt()', 'M()']


In [9]:
import pandas as pd
import root_pandas as rpd
reload(pd)
reload(rpd)
print "name"+utils.IO.signalName

signal_df = rpd.read_root_LorentzVector(utils.IO.signalName,"bbggSelectionTree", columns = branch_names, columns_lorentz=branch_names_lorentz,columns_lorentz_var=branch_names_\
lorentz_var)
background_df = rpd.read_root_LorentzVector(utils.IO.backgroundName,"bbggSelectionTree", columns = branch_names, columns_lorentz=branch_names_lorentz,columns_lorentz_var=branch_names_\
lorentz_var)
#example for converting files into csv. useful for plotting
signal_df.to_csv('/tmp/micheli/prova.csv',',',mode='w')

name/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170509/AllNonResSig.root
#################################################################################3
#################################################################################3


In [60]:
from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier(max_depth=5,learning_rate=0.2,n_estimators=100,min_weight_fraction_leaf=1e-4,verbose=1)
#clf = ensemble.AdaBoostClassifier(n_estimators=100)

utils.preprocessing.adjust_and_compress(signal_df).to_hdf('/tmp/micheli/signal.hd5','qcd',compression=9,complib='bzip2',mode='w')
utils.preprocessing.adjust_and_compress(background_df).to_hdf('/tmp/micheli/signal.hd5','qcd',compression=9,complib='bzip2',mode='w')

utils.preprocessing.define_process_weight(signal_df,1,1)
utils.preprocessing.define_process_weight(background_df,0,1)

y_sig = signal_df[['proc']]
y_bkg = background_df[['proc']]
y_total_train = utils.preprocessing.get_total_training_sample(y_sig,y_bkg).ravel()


X_sig= np.concatenate([signal_df[['leadingJet_bDis','subleadingJet_bDis']],
                             np.divide(signal_df[['diphotonCandidate.Pt()']],signal_df[['diHiggsCandidate.M()']]),
                             np.absolute(signal_df[['CosThetaStar_CS']]),
                             np.absolute(signal_df[['CosTheta_bb']])],
                             axis=1)
#X_sig_additional = np.concatenate([np.divide(signal_df[['diphotonCandidate.Pt()']],signal_df[['diHiggsCandidate.M()']])])

#X_bkg = background_df[['leadingJet_bDis','subleadingJet_bDis']]
X_bkg = np.concatenate([background_df[['leadingJet_bDis','subleadingJet_bDis']],
                             np.divide(background_df[['diphotonCandidate.Pt()']],background_df[['diHiggsCandidate.M()']]),
                             np.absolute(background_df[['CosThetaStar_CS']]),
                             np.absolute(background_df[['CosTheta_bb']])],
                             axis=1)


#X_total_train = utils.preprocessing.get_total_training_sample(X_sig,X_bkg)
X_total_train = utils.preprocessing.get_total_training_sample(pd.DataFrame(X_sig),pd.DataFrame(X_bkg))

#print X_total_train.size
print X_total_train.size

#print X_total_train
print X_total_train


6871625
[[ 0.91113281  0.99365234  0.63279173  0.17651367  0.44995117]
 [ 0.76269531  0.59814453  0.25899156  0.13085938  0.88818359]
 [ 0.81054688  0.58447266  0.47056144  0.25805664  0.33422852]
 ..., 
 [ 0.08734131  0.99316406  0.12315373  0.83544922  0.37304688]
 [ 0.47192383  0.15869141  0.169512    0.97460938  0.55908203]
 [ 0.35083008  0.54394531  0.29020381  0.49682617  0.14306641]]


In [55]:
X_totalissimo = np.concatenate([X_total_train,X_total_train_additional],axis=1)

X_totalissimo = np.append(X_total_train,X_total_train_additional,axis=1)

print "ciao"

ciao
